# SALDO DISPONÍVEL

Esta rotina passa os saldos de `BLOCKED` para `AVALIABLE`

In [ ]:
LIST_SOFIER_TARGET = [
    'fernandamatos417@gmail.com',
    'robertinhooj99@gmail.com',
    'katia_silva98@outlook.com',
    'luismiotta@hotmail.com',
    'mcris98@outlook.com.br',
    'pritybenites@gmail.com',
]

In [ ]:
import boto3

TABLE = boto3.session.Session(region_name='us-east-1', profile_name='sofie').resource('dynamodb').Table('table_sofier_ledger')
params = dict()

FINAL = list()

while True:
    response = TABLE.scan(**params)
    
    for each in response.get('Items'):
        if each['phase'] == 'BLOCKED' and each['sofier'] in LIST_SOFIER_TARGET:
            FINAL.append((each['sofier'], each['execution_id']))
    
    last_key = response.get('LastEvaluatedKey')
    if not last_key:
        break
        
    params['ExclusiveStartKey'] = last_key

from pprint import pprint
pprint(FINAL)

In [ ]:
from datetime import datetime

for sofier, execution_id in FINAL:
    TABLE.update_item(
        Key={'sofier': sofier, 'execution_id': execution_id},
        UpdateExpression='set phase = :ph, last_move = :lm, history = list_append(history, :h)',
        ExpressionAttributeValues={
            ':ph': 'AVAILABLE',
            ':lm': datetime.utcnow().isoformat(),
            ':h': [{'phase': 'AVAILABLE', 'time_stamp': datetime.utcnow().isoformat()}]
        }
    )

# RESGATE DE SALDOS

O objetivo deste _script_ é o de registrar os resgates artificialmente

In [9]:
VALORES = dict()

In [10]:
from uuid import uuid1
from decimal import Decimal
import boto3

TABLE = boto3.session.Session(region_name='us-east-1', profile_name='sofie').resource('dynamodb').Table('table_sofier_ledger')

for sofier, value in VALORES.items():
    item = {
        'sofier': sofier, 
        'execution_id': str(uuid1()),
        'reward': Decimal(-value),
        'phase': 'AVAILABLE',
        'last_move': datetime.utcnow().isoformat(),
        'comments': 'Pagamento efetuado diretamente pelo Erik ao grupo de testes - 2ª Leva',
        'dt_prevision': '2019-11-14T20:00:00'
    }
    
    TABLE.put_item(Item=item)
